<a href="https://colab.research.google.com/github/luucaslfs/soc_battle_dss/blob/main/Esoccerb_Funcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Mining/Cleaning



In [ ]:
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/America/Recife /etc/localtime
!date

Tue Oct  6 11:31:13 -03 2020


In [ ]:
# AJUSTANDO FUSO_HORARIO p/ RECIFE
# RODAR SEMPRE PRA CHECAR

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import csv

def tableDataText(table): #Retorna tabela em lista
    rows_output = []
    trs = table.find_all('tr')
    header_row = [td.get_text(strip=True) for td in trs[0].find_all('th')] # column_row
    header_row.append('MatchLink')
    rows_output.append(header_row)
    trs = trs[1:]
    for tr in trs: #pra cada linha na tabela
        lista = [td.get_text(strip=True) for td in tr.find_all('td')] # data_row
        for a in tr.find_all('a', href=True):
          if ('/r/') in a['href']: 
            lista.append(a['href'])
            link = 'https://www.betsapi.com' + a['href'] 
            print('link:' + link) 
            getFGTime(link)
        rows_output.append(lista)
    return rows_output

header = {'User-Agent': 'Mozilla/5.0'}

def getFGTime(link): #Captura tempo do primeiro gol(se houver)
    req = Request(link, None, headers=header)
    print('ok')
    page = urlopen(req)
    print(page)
    soup = BeautifulSoup(page, 'lxml')
    goals = soup.find('list-group')
    print(goals)

def tableRequest(link): #Captura tabela do link
    req = Request(link,headers=header)
    page = urlopen(req)
    soup = BeautifulSoup(page, 'lxml')
    table = soup.find('table')
    return table

def toDataFrame(link): #Retorna dataframe da tabela do link
    output = tableDataText(tableRequest(link))
    df_table = pd.DataFrame(output[1:], columns = output[0])
    return df_table

def get_url(page_number): #Recebe o mumero da pag(negativo) e retorna link
    return ('https://betsapi.com/le/22614/Esoccer-Battle--8-mins-play/p.' + str(-page_number))


outputs = pd.DataFrame() # Criando DF resultado c/ coluna de 1st.gt
page_number = 1000 # primeira pagina a ser capturada (da menos recente pra mais recente)

for page in range(-page_number, 0): # captura pgs de tras pra frente
    url = get_url(page)
    new_table = toDataFrame(url)
    outputs = outputs.append(new_table)
    #print('Pagina ' + str(-page) + ' agregada.')
    
outputs.to_csv('raw_table.csv') #salvando num csv
outputs

link:https://www.betsapi.com/r/2505208/Chelsea-%28KRaftVK%29-Esports-v-AC-Milan-%28MeLToSiK%29-Esports
ok


HTTPError: ignored

*Obs futura: 
1. Se no td tiver vazio(view), entrar no link e capturar placar.*
2. Pegar a primeira pagina e procurar 'ultima pagina'
3. Linha 0 do site é a ultima da pag




# Preprocessing

In [ ]:
import pandas as pd
import csv

df = pd.read_csv('raw_table.csv')

# Separando Times de 'Casa e Visitante' e 'Jogadores' de 'Times'
def splitTeams(df):
  # Caso em que 'Esportsv' separa os times
  df_case1 = df[df['Home v Away'].str.contains('Esportsv')].copy()
  df_case1_split = df_case1['Home v Away'].str.split('Esportsv', n=1, expand=True)
  
  df_case1_casa = df_case1_split[0].str.split(' \(', n=1, expand=True)
  df_case1_visitante = df_case1_split[1].str.split(' \(', n=1, expand=True)
  
  df_case1['Time A'] = df_case1_casa[0]
  df_case1['Time B'] = df_case1_visitante[0]
  df_case1['Jogador A'] = df_case1_casa[1].str.replace('\)', '')
  df_case1_jogadorb = df_case1_visitante[1].str.replace('\) Esports', '')
  df_case1['Jogador B'] = df_case1_jogadorb.str.replace('\)', '')
  return df_case1
  
  # Caso-contrario(s) TO-DO
  # 
  #
  #
  #

df = splitTeams(df)

# Reorganizando colunas e ajustando data/hora
# DATA SEMPRE pega pelo fuso do SERVIDOR/PC que roda isso aqui (checker lá em cima)
df = df.rename(columns={'-':'Resultado', 'Date':'Data'})
df = df.drop(columns=['Unnamed: 0', 'R', 'Home v Away'])
df['Data'] = '2021/' + df['Data']
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# lowercasing
columns_list = ['Time A', 'Time B', 'Jogador A', 'Jogador B']
for col in columns_list:
  df[col] = df[col].str.lower()

# Separando resultados dos dois times e deletando jogos 'Cancelled' e 'View'
inv_result_canceled = df[df['Resultado'] == 'Cancelled'] 
inv_result_view = df[df['Resultado'] == 'View']
inv_result = pd.concat([inv_result_canceled, inv_result_view]).index
df.drop(inv_result, inplace=True)

split_result = df['Resultado'].str.split('-', n=1, expand=True)
df['Placar A'] = split_result[0].astype('int32')
df['Placar B'] = split_result[1].astype('int32')

# Limpando Nulos
df = df.dropna()

# Casos Extras
df = df.rename(columns={'Data':'Date', 'Time A':'Team A', 'Time B':'Team B', 'Jogador A':'Player A', 'Jogador B':'Player B', 'Placar A':'Score A', 'Placar B':'Score B'})
df = df.drop(axis=1, columns='Resultado')
df

# Salvando csv
df.to_csv('prpc_table.csv')

#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#!cp prpc_table.csv "/content/gdrive/My Drive/Esoccerb/Data"

#ESTATISTICAS

In [ ]:
#ESTATISTICAS DE CONFRONTO

df_live = pd.read_csv('prpc_table.csv')
df_live = df_live.rename(columns={'Player A': 'PlayerA', 'Player B':'PlayerB', 'Score A':'ScoreA', 'Score B':'ScoreB'})

def calc_confronto(player1, player2): 
  match_1 = df_live[(df_live.PlayerA.str.contains(player1)) & (df_live.PlayerB.str.contains(player2))]
  match_2 = df_live[(df_live.PlayerA.str.contains(player2)) & (df_live.PlayerB.str.contains(player1))]
  confronto = pd.concat([match_1, match_2]).drop(axis=1, columns='Unnamed: 0')
  confronto_last = confronto.tail(10)

  pct_zeroazero = (len(confronto[(confronto.ScoreA == 0) & (confronto.ScoreB == 0)])*100)/len(confronto)
  pct_umaum = (len(confronto[(confronto.ScoreA == 1) & (confronto.ScoreB == 1)])*100)/len(confronto)
  pct_empates = (len(confronto[(confronto.ScoreA == confronto.ScoreB)])*100)/len(confronto)
  pct_ambos = (len(confronto[(confronto.ScoreA > 0) & (confronto.ScoreB > 0)])*100/len(confronto))

  pct_zeroazero_last = (len(confronto_last[(confronto_last.ScoreA == 0) & (confronto_last.ScoreB == 0)])*100)/len(confronto_last)
  pct_umaum_last = (len(confronto_last[(confronto_last.ScoreA == 1) & (confronto_last.ScoreB == 1)])*100)/len(confronto_last)
  pct_empates_last = (len(confronto_last[(confronto_last.ScoreA == confronto_last.ScoreB)])*100)/len(confronto_last)
  pct_ambos_last = (len(confronto_last[(confronto_last.ScoreA > 0) & (confronto_last.ScoreB > 0)])*100/len(confronto_last))

  vitorias_p1 = len(match_1[(match_1.ScoreA > match_1.ScoreB)]) + len(match_2[(match_2.ScoreA < match_2.ScoreB)])
  pct_p1 = (vitorias_p1*100)/len(confronto)
  vitorias_p2 = len(match_1[(match_1.ScoreA < match_1.ScoreB)]) + len(match_2[(match_2.ScoreA > match_2.ScoreB)]) 
  pct_p2 = (vitorias_p2*100)/len(confronto)

  print('\n\nConfronto: ' + player1 + '({0:.2f}%)'.format(pct_p1) + ' vs ' + player2 + '({0:.2f}%)'.format(pct_p2) + '\n')
  print('Vitorias de {}: {}'.format(player1, str(vitorias_p1)))
  print('Vitorias de {}: {}\n'.format(player2, str(vitorias_p2)))
  print('Jogos computados: '+ str(len(confronto)) + '\nEmpates: ' + format(pct_empates, '.2f')+'%' +'\n% do Total de Jogos Terminados em 0-0: '+ format(pct_zeroazero, '.2f') + '%' + '\n% do Total de Jogos Terminados em 1-1: ' + format(pct_umaum, '.2f') + '%\n' + '% de Ambos Marcam: ' + format(pct_ambos, '.2f') + '\n\nUltimos Resultados do Confronto: ')
  last_results = confronto # O numero em tail(#) indica a qtde de resultados que vai ser exibido
  last_results = last_results[['Date', 'Team A', 'PlayerA', 'ScoreA', 'ScoreB', 'PlayerB', 'Team B']]
  display(last_results.tail(10))
  print('\n\n ESTATISTICAS DOS ULTIMOS 10 CONFRONTOS\n' + '\nEmpates: ' + format(pct_empates_last, '.2f')+'%' +'\n% do Total de Jogos Terminados em 0-0: '+ format(pct_zeroazero_last, '.2f') + '%' + '\n% do Total de Jogos Terminados em 1-1: ' + format(pct_umaum_last, '.2f') + '%\n' + '% de Ambos Marcam: ' + format(pct_ambos_last, '.2f'))
  
def playerstats(player1):
  player_base = ['upcake22', 'meltosik', 'labotryas', 'kray', 'kraftvk']
  for player2 in player_base:
    if (player2 == player1) == False:
      calc_confronto(player1, player2)


In [ ]:
# RODE ESTA CELULA PRA CALCULAR AS ESTATISTICAS DO CONFRONTO

#playerstats('upcake22')
calc_confronto('olle', 'inq')
#calc_confronto('up', 'lab')
# Resultado abaixo




Confronto: olle(0.00%) vs inq(100.00%)

Vitorias de olle: 0
Vitorias de inq: 1

Jogos computados: 1
Empates: 0.00%
% do Total de Jogos Terminados em 0-0: 0.00%
% do Total de Jogos Terminados em 1-1: 0.00%
% de Ambos Marcam: 0.00

Ultimos Resultados do Confronto: 


,Date,Team A,PlayerA,ScoreA,ScoreB,PlayerB,Team B
27,2020-10-06 12:40:00,psg,inquisitor,1,0,olle,man city




 ESTATISTICAS DOS ULTIMOS 10 CONFRONTOS

Empates: 0.00%
% do Total de Jogos Terminados em 0-0: 0.00%
% do Total de Jogos Terminados em 1-1: 0.00%
% de Ambos Marcam: 0.00


In [ ]:
df_live.head(10)

,Unnamed: 0,Date,Team A,Team B,PlayerA,PlayerB,ScoreA,ScoreB
0,0,2020-07-16 16:48:00,crystal palace,leicester,blackstar98,salayyy,2,1
1,1,2020-07-16 16:48:00,wolverhampton,west ham,taka,flewless_phoenix,3,0
2,2,2020-07-16 16:42:00,real madrid,liverpool,olle,inquisitor,0,0
3,3,2020-07-16 16:42:00,psg,barcelona,pablik,orlovsky1,1,2
4,4,2020-07-16 16:36:00,everton,wolverhampton,chellovekk,taka,0,1
5,5,2020-07-16 16:36:00,west ham,crystal palace,flewless_phoenix,blackstar98,2,2
6,6,2020-07-16 16:30:00,man city,real madrid,dm1trena,olle,0,0
7,7,2020-07-16 16:30:00,liverpool,psg,inquisitor,pablik,2,0
8,8,2020-07-16 16:24:00,leicester,west ham,salayyy,flewless_phoenix,0,1
9,9,2020-07-16 16:24:00,crystal palace,everton,blackstar98,chellovekk,0,1
